<a href="https://www.kaggle.com/code/tayen1/nyc-taxi-duration?scriptVersionId=236903184" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/tayen1/nyc-taxi-duration?scriptVersionId=235741972" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/tayen1/nyc-taxi-duration?scriptVersionId=235737872" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/tayen1/nyc-taxi-duration?scriptVersionId=235734476" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,median_absolute_error,r2_score,explained_variance_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

output_dir = '/kaggle/working/'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data_path_name = os.path.join(dirname, filename)
        print(data_path_name)
        with zipfile.ZipFile(data_path_name, 'r') as zip_ref:
            zip_ref.extractall(output_dir)

print("Finish unzip")

for dirname, _, filenames in os.walk(output_dir):
    for filename in filenames:
        print(os.path.join(output_dir, filename))

/kaggle/input/nyc-taxi-trip-duration/train.zip
/kaggle/input/nyc-taxi-trip-duration/test.zip
/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip
Finish unzip
/kaggle/working/sample_submission.csv
/kaggle/working/train.csv
/kaggle/working/test.csv


In [2]:
test_df = pd.read_csv("/kaggle/working/test.csv")
train_df = pd.read_csv("/kaggle/working/train.csv")
sample_submission_df = pd.read_csv("/kaggle/working/sample_submission.csv")

In [3]:
train_df

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373


In [4]:
test_df

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N
...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N


In [5]:
sample_submission_df

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959
...,...,...
625129,id3008929,959
625130,id3700764,959
625131,id2568735,959
625132,id1384355,959


In [4]:
print(train_df.isnull().sum())
print(test_df.isnull().sum())

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64
id                    0
vendor_id             0
pickup_datetime       0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
dtype: int64


In [7]:
print(train_df.duplicated().sum())
print(test_df.duplicated().sum())

0
0


In [8]:
train_df.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [ ]:
train_rf_df = train_df.copy()
test_rf_df = test_df.copy()

In [ ]:
#Obtain the day of the week and the hour of pickup from the train data
train_rf_df['pickup_datetime'] = pd.to_datetime(train_rf_df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
train_rf_df['pickup_week'] = train_rf_df['pickup_datetime'].dt.day_name()
train_rf_df['pickup_hour'] = train_rf_df['pickup_datetime'].dt.hour
#Obtain the day of the week and the hour of pickup from the test data
test_rf_df['pickup_datetime'] = pd.to_datetime(test_rf_df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
test_rf_df['pickup_week'] = test_rf_df['pickup_datetime'].dt.day_name()
test_rf_df['pickup_hour'] = test_rf_df['pickup_datetime'].dt.hour

In [5]:
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371.0
    q = np.radians(lat1)
    w = np.radians(lat2)
    e = np.radians(lat2 - lat1)
    r = np.radians(lon2 - lon1)
    
    a = np.sin(e/2.0)**2 + np.cos(q) * np.cos(w) * np.sin(r/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [ ]:
#Get the distace from point to point
train_rf_df['distance_km'] = haversine_np(train_rf_df['pickup_latitude'],
    train_rf_df['pickup_longitude'],
    train_rf_df['dropoff_latitude'],
    train_rf_df['dropoff_longitude'])
#Get the distace from point to point
test_rf_df['distance_km'] = haversine_np(test_rf_df['pickup_latitude'],
    test_rf_df['pickup_longitude'],
    test_rf_df['dropoff_latitude'],
    test_rf_df['dropoff_longitude'])

In [ ]:
train_rf_df.head()
test_rf_df.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_week,pickup_hour,distance_km
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,Thursday,23,2.746426
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,Thursday,23,2.759239
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,Thursday,23,1.306155
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,Thursday,23,5.269088
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,Thursday,23,0.960842


In [ ]:
#Observe datatype of the columns
print(train_rf_df.dtypes)
print(test_rf_df.dtypes)

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime              object
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                  int64
pickup_week                   object
pickup_hour                    int32
distance_km                  float64
dtype: object
id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
pickup_week                   object
pickup_hour                    int32
distance_km                  float64
dtype: object


In [8]:
#Define the independent variables in categories 
cat_cols = ['vendor_id', 'store_and_fwd_flag', 'pickup_week']
num_cols   = ['passenger_count','pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude','pickup_hour', 'distance_km']

In [ ]:

train_rf_df['log_trip_duration'] = np.log1p(train_rf_df['trip_duration'])

In [ ]:
X = train_rf_df[num_cols + cat_cols]
y = train_rf_df['log_trip_duration']
X_test  = test_rf_df[num_cols + cat_cols]
x_train,x_val,y_train,y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
#Set up a preprocessing pipeline
preprocessor = ColumnTransformer(transformers=[ ('num', StandardScaler(), num_cols),('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols)])

In [12]:
X_train_proc = preprocessor.fit_transform(x_train)
X_test_proc  = preprocessor.transform(x_val)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
RF = RandomForestRegressor(n_estimators=50, random_state=42)
RF.fit(X_train_proc, y_train)
P_RF = RF.predict(X_test_proc)



In [15]:
metrics = {
    'MSE (Actual value)'      : mean_squared_error(np.expm1(y_val), np.expm1(P_RF)),
    'MAPE'     : mean_absolute_percentage_error(y_val, P_RF),
    'MedianAE' : median_absolute_error(y_val, P_RF),
    'R2'       : r2_score(y_val, P_RF),
    'ExplVar'  : explained_variance_score(y_val, P_RF),
    "rmsle" : np.sqrt(mean_squared_error(y_val, P_RF ))
}
for name, val in metrics.items():
    print(f"{name:7s}: {val:.4f}")

MSE (Actual value): 10255484.6966
MAPE   : 0.0428
MedianAE: 0.1875
R2     : 0.7355
ExplVar: 0.7356
rmsle  : 0.4097
